In [1]:
import numpy as np
import pymc as pm
import arviz as az

from astropy.table import Table

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
# Load the fits file
data = Table.read('StellarMassesv19.fits', format='fits')

#print (data.colnames)
filter_data = data[(data['logmstar'] + np.log10(data['fluxscale']) > 9) & (data['Z'] > 0.035) & (data['Z'] < 0.065) & (data['fluxscale'] > 0.3) & (data['fluxscale'] < 3)]

fluxscale = filter_data['fluxscale']
z = filter_data['Z']
logM = filter_data['logmstar']+np.log10(fluxscale)
dlogM = filter_data['dellogmstar']

C:\Users\catch\AppData\Local\Temp\ipykernel_5544\3860482609.py:5: RuntimeWarning: invalid value encountered in log10
  filter_data = data[(data['logmstar'] + np.log10(data['fluxscale']) > 9) & (data['Z'] > 0.035) & (data['Z'] < 0.065) & (data['fluxscale'] > 0.3) & (data['fluxscale'] < 3)]


In [6]:
from sympy import uppergamma, N
from scipy.integrate import quad

def comoving_volume(z_initial, z_final, H0=70, Omega_M=0.3, Omega_Lambda=0.7):
    c = 3e5 #light speed in km/s as H0 is 70km/s/MPc
    # For a flat universe, Omega_k = 0
    Omega_k = 1 - Omega_M - Omega_Lambda  
    # Hubble distance in Mpc
    DH = c / H0  

    # Angular size distance
    def E(z, Omega_M=Omega_M, Omega_Lambda=Omega_Lambda, Omega_k=Omega_k):
        return np.sqrt(Omega_M * (1 + z)**3 + Omega_k * (1 + z)**2 + Omega_Lambda)

    # Integral for comoving distance DC
    def comoving_distance(z):
        return quad(lambda z: 1/E(z), 0, z)[0] * DH

    # Calculate DC for initial and final redshifts
    DC_initial = comoving_distance(z_initial)
    DC_final = comoving_distance(z_final)

    # Volume of the light cone V_C using the formula for Omega_k = 0 (flat universe)
    VC_initial = (4 * np.pi / 3) * DC_initial**3
    VC_final = (4 * np.pi / 3) * DC_final**3

    # The volume covered between z_initial and z_final is the difference
    V_360 = VC_final - VC_initial
    
    #full sky has 41253 sq degrees and GAMA has 250sq degrees
    V_gama = 143/41253 * V_360
    
    return V_gama

# Initial and final redshifts
z_initial = 1e-6
z_final = 0.065

# Calculate and print the volume of the light cone
volume = comoving_volume(z_initial, z_final)
print(f"The volume of the light cone between z={z_initial} and z={z_final} is approximately {volume:.2f} cubic Mpc.")

# Schechter function definition
def schechter_func(parvals, data):
    alpha, log_m_star, phi_star= parvals
    log_mass= data
    x = log_mass  - log_m_star
    sch = np.log(10) * phi_star * (10 ** (x * (alpha + 1))) * np.exp(-10 ** x)
    sch = np.where(np.isfinite(sch), sch, 0)
    return sch

def schechter_int(parvals, data):
    alpha, log_m_star, phi_star= parvals
    log_mass= data 
    [a, b] = [min(log_mass) - log_m_star, max(log_mass) - log_m_star]

    # Compute uppergamma values using sympy's N to ensure numerical values
    gamma1_a = float(N(uppergamma(alpha + 1, 10**a)))
    gamma1_b = float(N(uppergamma(alpha + 1, 10**b)))

    norm = ((phi_star) * (gamma1_a - gamma1_b))
    return(norm)

The volume of the light cone between z=1e-06 and z=0.065 is approximately 300145.55 cubic Mpc.


In [13]:
from sympy.printing.theanocode import theano_function

# Convert the sympy expression to a Theano function
f_theano = theano_function(parvals, schechter_func)

def schechter(phi_star, L_star, L, alpha):
    return f_theano(phi_star, L_star, L, alpha)

AttributeError: partially initialized module 'theano' has no attribute 'compile' (most likely due to a circular import)

In [8]:
# PyMC model definition


with pm.Model() as model:
    # Priors for the parameters
    alpha = pm.Normal('alpha', mu=-1, sigma=1.5)
    ln_m_star = pm.Normal('ln_m_star', mu=9.7, sigma=0.005)    
    phi_star = pm.Normal('phi_star', mu=30, sigma=10)

    parvals = [alpha,ln_m_star, phi_star]
    
    # Calculate the Schechter function values
    sch = schechter_func(parvals, logM)
    
    # Integrate for normalization
    norm1 = schechter_int(parvals, logM)
    
    # Likelihood
    const = 1e-10  # To avoid log(0)
    likelihood = pm.Normal('likelihood', mu=sch / norm, sigma=const, observed=np.ones_like(ln_mass_data))
    
    # Inference
    trace = pm.sample(1000, tune=1000, chains=4)

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''